导入相应的库
--

In [1]:
import os

import numpy as np
import paddle
import pandas as pd
import scipy
import copy

from collections import OrderedDict
from matplotlib import pyplot as plt
from numpy import diag
from paddle import fluid
from paddle.complex import kron, matmul, trace
from paddle_quantum.circuit import UAnsatz
from paddle_quantum.utils import dagger, partial_trace, state_fidelity
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from tqdm import tqdm

数据处理
--

In [2]:
#root = '/home/johnson-lin/Downloads'
#npz_file = np.load(os.path.join(root, 'breastmnist.npz'))
npz_file = np.load('minidata.npz')

In [3]:
train_images = npz_file['train_images']
train_images = train_images.reshape(train_images.shape[0], -1)

val_images = npz_file['val_images']
val_images = val_images.reshape(val_images.shape[0], -1)

pca = PCA(n_components=8)
new_train = pca.fit_transform(train_images)
new_val = pca.fit_transform(val_images)

In [4]:
# 保证每张图片的向量所有元素之和为1
for i in range(len(new_train)):
    new_train[i] = new_train[i] / new_train[i].sum()
    
for i in range(len(new_val)):
    new_val[i] = new_val[i] / new_val[i].sum()

电路设置
--

In [5]:
N_A = 2        # 系统 A 的量子比特数
N_B = 1        # 系统 B 的量子比特数
N = N_A + N_B  # 总的量子比特数

scipy.random.seed(1)                            # 固定随机种子
V = scipy.stats.unitary_group.rvs(2**N)         # 随机生成一个酉矩阵
V_H = V.conj().T                                # 进行厄尔米特转置


In [6]:
# 设置电路参数
cir_depth = 6                        # 电路深度
block_len = 2                        # 每个模组的长度
theta_size = N*block_len*cir_depth   # 网络参数 theta 的大小


# 搭建编码器 Encoder E
def Encoder(theta):

    # 用 UAnsatz 初始化网络
    cir = UAnsatz(N)
    
    # 搭建层级结构：
    for layer_num in range(cir_depth):
        
        for which_qubit in range(N):
            cir.ry(theta[block_len*layer_num*N + which_qubit], which_qubit)
            cir.rz(theta[(block_len*layer_num + 1)*N + which_qubit], which_qubit)

        for which_qubit in range(N-1):
            cir.cnot([which_qubit, which_qubit + 1])
        cir.cnot([N-1, 0])

    return cir.U

In [7]:
def normalize2unitary(x):
    rho_in_mols=x
    rho_in_mols=(V@diag(rho_in_mols)@V_H).astype('complex128')
    return rho_in_mols

In [8]:
def top_k_sum(arr, k):
    top_k_idx = arr.argsort()[::-1][0:k]
    top_k_sum = 0
    for idx in top_k_idx:
        top_k_sum += arr[idx]
    return top_k_sum

In [9]:
rho_C = np.diag([1,0]).astype('complex128')

In [10]:
def plot_curve(loss, fid):
    plt.xlabel("epochs")
    plt.ylabel("loss")
    plt.plot(loss, label='train loss', marker="s")
    plt.legend(loc='best')
    plt.grid()
    plt.savefig('AE_train_ls.png')
    plt.close()
    plt.xlabel("epochs")
    plt.ylabel("fid")
    plt.plot(fid, label='train fid', marker="s")
    plt.legend(loc='best')
    plt.grid()
    plt.savefig('AE_train_fid.png')
    plt.close()
    

量子网络搭建
--

In [11]:
N_A = 2        # 系统 A 的量子比特数
N_B = 1        # 系统 B 的量子比特数
N = N_A + N_B  # 总的量子比特数
# ITR = 100      # 设置迭代次数
SEED = 14      # 固定初始化参数用的随机数种子

class NET4(fluid.dygraph.Layer):
    """
    Construct the model net
    """
    def __init__(self, shape, param_attr=fluid.initializer.Uniform(
        low=0.0, high=2 * np.pi, seed = SEED), dtype='float64'):
        super(NET4, self).__init__()
        
        # 我们需要将 Numpy array 转换成 Paddle 动态图模式中支持的 variable
        self.rho_C = fluid.dygraph.to_variable(rho_C)
        self.theta = self.create_parameter(shape=shape, 
                     attr=param_attr, dtype=dtype, is_bias=False)
    
    # 定义损失函数和前向传播机制
    def forward(self,x):
        # 生成初始的编码器 E 和解码器 D\n",
        rho_in= fluid.dygraph.to_variable(x)
        E = Encoder(self.theta)
        E_dagger = dagger(E)
        D = E_dagger
        D_dagger = E

        # 编码量子态 rho_in
        rho_BA = matmul(matmul(E, rho_in), E_dagger)
        
        # 取 partial_trace() 获得 rho_encode 与 rho_trash
        rho_encode = partial_trace(rho_BA, 2 ** N_B, 2 ** N_A, 1)
        rho_trash = partial_trace(rho_BA, 2 ** N_B, 2 ** N_A, 2)

        # 解码得到量子态 rho_out
        rho_CA = kron(self.rho_C, rho_encode)
        rho_out = matmul(matmul(D, rho_CA), D_dagger)
        
        # 通过 rho_trash 计算损失函数
        
        zero_Hamiltonian = fluid.dygraph.to_variable(np.diag([1,0]).astype('complex128'))
        loss = 1 - (trace(matmul(zero_Hamiltonian, rho_trash))).real

        return loss, rho_out

训练
--

In [ ]:
LR = 0.1       # 设置学习速率
EPOCHS = 5

with fluid.dygraph.guard():
    net = NET4([theta_size])

    opt = fluid.optimizer.AdagradOptimizer(learning_rate=LR,
                          parameter_list=net.parameters())

    tr_fid = []
    tr_ls = []
    best_fid = 0
    
    for epoch in range(EPOCHS):
        epoch_fid = []
        epoch_ls = []
        for i in tqdm(range(len((new_train)))):
            x=new_train[i]
            s=top_k_sum(x, 2**N_A)
            trainx=normalize2unitary(x)
            loss, rho_out=net(trainx)

            loss.backward()
            opt.minimize(loss)
            net.clear_gradients()
            fid=state_fidelity(trainx, rho_out.numpy()) / s
            epoch_fid.append(fid)
            epoch_ls.append(loss.numpy())
        tr_fid.append(np.square(np.array(epoch_fid).mean()))
        tr_ls.append(np.array(epoch_ls).mean())
        
        if best_fid < np.square(np.array(epoch_fid).mean()):
            best_fid=np.square(np.array(epoch_fid).mean())
            fluid.save_dygraph(net.state_dict(), "paddle_dy")

        print('epoch:', epoch, 'loss:', '%.4f' % np.array(epoch_ls).mean(),
              'fid:', '%.4f' % np.square(np.array(epoch_fid).mean()))
    plot_curve(tr_ls, tr_fid)
    print('-'*30, 'TEST', '-'*30)
    para_state_dict, _ = fluid.load_dygraph("paddle_dy")
    net.set_dict(para_state_dict)
    test_fid = []
    test_ls = []
    for i in tqdm(range(len((new_val)))):
        x=new_val[i]
        s=top_k_sum(x, 2**N_A)
        trainx=normalize2unitary(x)
        loss, rho_out=net(trainx)

        #loss.backward()
        #opt.minimize(loss)
        #net.clear_gradients()
        fid=state_fidelity(trainx, rho_out.numpy()) / s
        test_fid.append(fid)
        test_ls.append(loss.numpy())
    print('test fid: ', np.square(np.array(test_fid).mean()))
    print('test loss: ', np.array(test_ls).mean())

  0%|          | 1/1622 [00:00<02:46,  9.74it/s]

epoch: 0 loss: -0.7165 fid: 0.7750


  0%|          | 2/1622 [00:00<02:29, 10.80it/s]

epoch: 1 loss: -1.2913 fid: 0.7333


  0%|          | 1/1622 [00:00<02:46,  9.73it/s]

epoch: 2 loss: -1.4886 fid: 0.7034


 53%|█████▎    | 866/1622 [01:22<01:21,  9.23it/s]